In [ ]:
import pandas as pd
import numpy as np
import math
import csv
import glob
import re
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import itertools
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from natsort import natsorted
import sklearn
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA

In [ ]:
class Data:
    def __init__(self, path):
        self.path = path
        self.dirs = natsorted(os.listdir(self.path))

    def get_files(self):
        files_list = []
        fileslength = []
        for f1 in self.dirs:
            # 対象がフォルダである場合にTrueが返り、if文内が実行される
            if os.path.isdir(os.path.join(self.path, f1)):
                for f2 in os.listdir(os.path.join(self.path, f1)):
                    if os.path.isdir(os.path.join(self.path, f1, f2)):
                        files = list(
                            natsorted(
                                glob.glob(os.path.join(self.path, f1, f2, "*.irr"))
                            )
                        )
                        if bool(files) == True:  # 空リスト除外
                            files_list.append(files[:-1])
                            fileslength.append(len(files[:-1]))
                else:
                    files = list(
                        natsorted(glob.glob(os.path.join(cur_dir, f1, "*.irr")))
                    )
                    if bool(files) == True:  # 空リスト除外
                        files_list.append(files[:-1])
                        fileslength.append(len(files[:-1]))
        fileslength = [x for x in fileslength if x != 0]
        return files_list, fileslength

    def get_dataframe(self):
        files_list, fileslength = self.get_files()
        df_list = []
        AREA = []
        for n, files in enumerate(files_list):
            dfs = []
            for i, file in enumerate(files):
                DF = pd.read_csv(file, header=None, sep=" ", skiprows=1)
                AREA.append(sum(DF.iloc[:, 3]))
                # dfs.append(DF.iloc[:,3] / AREA[n])
                dfs.append(DF.iloc[:, 3])

            df = pd.concat(dfs, axis=1)
            df = df.T
            df = df.rename(columns=lambda s: 1 / (271 + 0.5 * s) * 10**7)
            df = df.reset_index(drop=True)
            df = df.fillna(0)
            df_list.append(df)
        return df_list, AREA


def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


class CreateZ:
    def __init__(self, fileslength, x_step, x_distance, y_distance):
        self.fileslength = fileslength
        self.x_step = x_step
        self.x_distance = x_distance
        self.y_distance = y_distance
        self.y_step = [int(i / x_step) for i in fileslength]

    def meshgrid(self):
        X_list = []
        Y_list = []
        for n, length in enumerate(self.fileslength):
            x = np.arange(0, self.y_step[n] * self.y_distance, self.y_distance)
            y = np.arange(0, self.x_step * self.x_distance, self.x_distance)
            X, Y = np.meshgrid(x, y)
            X_list.append(X)
            Y_list.append(Y)
        return X_list, Y_list

    def z_intensity(self, df_list, av_spec, X_list, Y_list, min_AREA, min_sim):
        x_step = self.x_step
        y_step = self.y_step
        Z_list = []
        for n in range(len(X_list)):
            Z = np.zeros([x_step, y_step[n]])
            for i in range(x_step):
                for j in range(y_step[n]):
                    if (
                        sum(df_list[n].iloc[i + j * x_step]) < min_AREA
                        or cos_sim(df_list[n].iloc[i + j * x_step], av_spec) < min_sim
                    ):  # スペクトル強度とcos類似度の閾値以下のスペクトルは除外
                        Z[i, j] = np.nan
                    else:
                        Z[i, j] = sum(df_list[n].iloc[i + j * x_step])
            Z_list.append(Z)
        return Z_list

    def z_phi(self, df_list, av_spec, X_list, Y_list, phi, min_AREA, min_sim):
        x_step = self.x_step
        y_step = self.y_step
        Z_phi_list = []
        for n in range(len(X_list)):
            Z = np.zeros([x_step, y_step[n]])
            for i in range(x_step):
                for j in range(y_step[n]):
                    if (
                        sum(df_list[n].iloc[i + j * x_step]) < min_AREA
                        or cos_sim(df_list[n].iloc[i + j * x_step], av_spec) < min_sim
                    ):  # スペクトルなしと変なスペクトル除外
                        Z[i, j] = np.nan
                    else:
                        Z[i, j] = phi_list[n][i + j * x_step]
            Z_phi_list.append(Z)
        return Z_phi_list


def mesh_plot(X, Y, Z, label, dirs, N=False, vmin=[0], vmax=[1]):
    fig = plt.figure(figsize=(25, 4 * len(X)))
    plt.subplots_adjust(wspace=0.05)
    for i in range(len(X)):
        ax = fig.add_subplot(len(X), 3, i + 1)
        if N == False:
            norm = Normalize(
                vmin=min(Z[i].reshape(-1, 1))[0], vmax=max(Z[i].reshape(-1, 1))[0]
            )
        else:
            norm = Normalize(vmin=vmin[i], vmax=vmax[i])
        ax.pcolormesh(
            X[i],
            Y[i],
            Z[i],
            cmap="rainbow",
            shading="gouraud",
            norm=norm,
        )
        ax.set_title(dirs[i], size=20)
        ax.set_xticks(np.arange(0, len(Y[i][0]) * y_distance, 2))
        ax.grid(color="black")
        fig.colorbar(ScalarMappable(cmap="rainbow", norm=norm), ax=ax).set_label(
            label, size=20
        )
    plt.show()


class Chemometrics:
    def __init__(self, df):
        self.df = df

    def NMF(self, components):
        global model
        model = NMF(n_components=components, max_iter=1000).fit(self.df)
        loading = pd.DataFrame(model.components_, columns=self.df.columns)
        score = pd.DataFrame(model.transform(self.df), index=self.df.index)
        loading.columns = loading.columns.astype(float)

        # loadingを面積1に規格化
        for i in range(len(loading)):
            A = loading.iloc[i].sum()
            loading.iloc[i] = loading.iloc[i] / A
            score.iloc[:, i] = score.iloc[:, i] * A

        return loading, score


def calculation_phi(loading, score, N):  # N:エキシマー発光の成分
    excimer_area = [sum(loading.iloc[N] * score[N].iloc[i]) for i in range(len(score))]
    re_spec = np.dot(score, loading)
    all_area = [sum(re_spec[i]) for i in range(len(re_spec))]
    phi = [x / y for x, y in zip(excimer_area, all_area)]
    return phi

In [ ]:
# cur_dir = os.getcwd()  # カレントディレクトリ取得
cur_dir = "/Users/yusuke/Desktop/解析/mapping/0714(混練ラウンドノッチ)"
dirs = natsorted(os.listdir(cur_dir))
dirs

In [ ]:
files_list, fileslength = Data(cur_dir).get_files()
files_list, fileslength

In [ ]:
df_list, AREA = Data(cur_dir).get_dataframe()
df_list

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 4))

av_spec = pd.concat(df_list).mean()  # 平均スペクトル

sim = []
for n, df in enumerate(df_list):
    for i in range(0, len(df), 10):
        ax[0].plot(df.iloc[i])
        sim.append(cos_sim(df.iloc[i], av_spec))
sim_av = np.average([x for x in sim if math.isnan(x) == False])  # cos_simの平均値

ax[0].plot(av_spec, color="black", lw=3)
ax[0].set_xlim(23000, 17000)
ax[1].plot(sim, marker="o", linestyle="None")
ax[1].hlines(sim_av, 0, len(sim), "black", linestyles="dashed")
ax[1].set_ylabel("cos_sim")
ax[2].plot(AREA)
ax[2].hlines(np.average(AREA), 0, len(AREA), "black", linestyles="dashed")
ax[2].set_ylabel("Sum of Intensity")
print(sim_av)

In [ ]:
x_step = 16
x_distance = 0.5
y_distance = 0.5

Z = CreateZ(fileslength, x_step, x_distance, y_distance)

X_list, Y_list = Z.meshgrid()

Z_intensity = Z.z_intensity(df_list, av_spec, X_list, Y_list, min_AREA=1, min_sim=0.5)

In [ ]:
mesh_plot(X_list, Y_list, Z_intensity, "Intensity", dirs)

In [ ]:
df_all = pd.concat(df_list)

loading, score = Chemometrics(df_all).NMF(2)

fig, ax = plt.subplots(1, 2, figsize=(15, 3))
loading.T.plot(ax=ax[0], ylabel="Loadig", xlim=(23000, 16000))
score.plot(ax=ax[1], ylabel="Score", marker="o", linestyle="None", fillstyle="none")
plt.show()

In [ ]:
loading_list = []
score_list = []
for i in range(len(df_list)):
    loading_list.append(loading)
    if i == 0:
        score_list.append(score.iloc[0 : fileslength[i]])
    else:
        score_list.append(score.iloc[sum(fileslength[:i]) : sum(fileslength[: i + 1])])

phi_list = []
for n in range(len(loading_list)):
    phi_list.append(calculation_phi(loading_list[n], score_list[n], 1))

In [ ]:
Z_phi = Z.z_phi(df_list, av_spec, X_list, Y_list, phi_list, min_AREA=1, min_sim=0.5)

mesh_plot(X_list, Y_list, Z_phi, "phi", dirs[1:])

In [ ]:
Y_center = [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]  # 中心のY座標
Y_center2 = [int(x / y_distance) for x in Y_center]  # distance考慮

fig, ax = plt.subplots(4, 3, figsize=(20, 10))
plt.subplots_adjust(wspace=0.3, hspace=0.5)

dfc = []
for n, Z in enumerate(Z_phi):
    dfc.append(Z[Y_center2[n]])

    ax[n // 3, n % 3].scatter(
        [x * x_distance for x in range(len(Z[Y_center2[n]]))], Z[Y_center2[n]]
    )
    ax[n // 3, n % 3].set(
        ylim=(0, 1), title="{0}".format(dirs[n]), xlabel="X", ylabel="phi"
    )
plt.show()